# Load and Normalize Data


In [1]:
# Imports
import numpy as np
import pandas as pd
import re

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

## Create DataFrame from TEP Count Matrix

In [2]:
# Read data file into a panda dataframe
file_in = 'TEP_Count_Matrix.tsv.gz'
df_data = pd.read_csv(file_in, sep='\t')
print(df_data.shape)   # 5440 genes, 2351 samples
df_data.head(5)

(5440, 2351)


,1-Vumc-HD-101-TR922,2-Vumc-HD-103-TR923,3-Vumc-HD-108-TR924,4-Vumc-HD-127-TR925,5-Vumc-HD-130-TR926,6-Vumc-HD-142-TR927,7-Vumc-HD-143-TR928,8-Vumc-HD-144-TR929,9-Vumc-HD-145-TR930,11-Vumc-HD-149-TR932,...,countMatrix.3502-NKI-4854-SARC-TR3080,countMatrix.3503-NKI-4707-SARC-TR3079,countMatrix.3504-NKI-4941-SARC-TR3084,countMatrix.3507-NKI-6348-SARC-TR3134,countMatrix.3508-NKI-5275-SARC-TR3099,countMatrix.3512-NKI-5148-SARC-TR3129,countMatrix.320-10,countMatrix.327-34,countMatrix.376-P-PH-001-013-TR1400,countMatrix.506-NKI-NSCLC-107-270
ENSG00000000419,5,79,58,128,93,12,0,0,18,28,...,3,0,1,2,9,16,22,21,0,1
ENSG00000000460,8,10,21,14,6,8,25,42,5,7,...,0,3,0,2,63,3,3,0,1,0
ENSG00000000938,24,336,334,59,120,22,31,13,19,36,...,8,45,1,7,163,90,46,75,14,8
ENSG00000001036,38,46,15,36,25,43,27,27,59,5,...,1,3,4,15,3,6,10,8,3,0
ENSG00000001461,72,17,11,9,18,28,5,6,49,18,...,1,1,0,0,12,2,6,6,1,0


In [3]:
list_of_genes = df_data.index
list_of_genes

Index(['ENSG00000000419', 'ENSG00000000460', 'ENSG00000000938',
       'ENSG00000001036', 'ENSG00000001461', 'ENSG00000001497',
       'ENSG00000001629', 'ENSG00000001631', 'ENSG00000002330',
       'ENSG00000002549',
       ...
       'ENSG00000267279', 'ENSG00000267316', 'ENSG00000268555',
       'ENSG00000269028', 'ENSG00000270055', 'ENSG00000271043',
       'ENSG00000272053', 'ENSG00000272168', 'ENSG00000272369',
       'ENSG00000272888'],
      dtype='object', length=5440)

## Normalize TEP Count Matrix

### Look for gene length in GFF3

In [4]:
# Read GFF3
file='GFF3.csv'
df_GFF3 = pd.read_csv(file,sep=';',na_values='n.a.',index_col=0)
print(df_GFF3.shape)
df_GFF3.head(5)

(196520, 2)


,gene,Length
ENST00000456328,ENSG00000223972,1657
ENST00000515242,ENSG00000223972,1653
ENST00000518655,ENSG00000223972,1483
ENST00000450305,ENSG00000223972,632
ENST00000438504,ENSG00000227232,1783


In [5]:
# Check how many transcripts we have per gene
df_GFF3_transcripts_per_gene = df_GFF3.groupby('gene').count()
df_GFF3_transcripts_per_gene = df_GFF3_transcripts_per_gene.rename(columns = {'Length':'number_of_transcripts'})
df_GFF3_transcripts_per_gene.describe()
# df_GFF3_transcripts_per_gene.head(3)

,number_of_transcripts
count,57820.000000
mean,3.398824
std,4.880759
min,1.000000
25%,1.000000
50%,1.000000
75%,4.000000
max,82.000000


Each gene has between 1 and 82 transcripts. 

In the pacients database we only have the name of the gene, but not the name of the transcript measured. Therefore, we need to reduce number of transcripts per gene to have a more accurate Length number to use in the normalization.

### Look for gene length finding most common isoform per gene

In [6]:
# Load Best Transcript file
file='BestTranscript.csv'
df_best_transcripts = pd.read_csv(file, sep=';')
print("Size: ", df_best_transcripts.shape)
df_best_transcripts.head(5)

Size:  (25508, 4)


,name,transcript,gene,Length
0,uc010nxq.1,ENST00000518655,ENSG00000223972,1483
1,uc009viu.3,ENST00000438504,ENSG00000227232,1783
2,uc001aak.3,ENST00000417324,ENSG00000237613,1187
3,uc001aal.1,ENST00000335137,ENSG00000186092,918
4,uc021oeg.2,ENST00000493797,ENSG00000239906,323


In [7]:
# # 3 cases
# a) Genes that appear only once in df_best_transcripts
# b) Genes that appear MORE THAN once in df_best_transcripts
# c) Genes that DO NOT appear in df_best_transcripts


In [8]:
# Create a dataframe with those best transcripts whose gene appear in the patients dataframe
df_best_transcripts_subset = df_best_transcripts[df_best_transcripts['gene'].isin(list_of_genes)]
df_best_transcripts_subset.shape

(5454, 4)

In [9]:
# a) Genes that appear only once in df_best_transcripts
df_best_transcripts_subset_once = df_best_transcripts_subset.drop_duplicates(subset=['gene'], keep=False, inplace=False)

print("Size: ", df_best_transcripts_subset_once.shape)
df_best_transcripts_subset_once.head()

Size:  (5251, 4)


,name,transcript,gene,Length
16,uc001abz.4,ENST00000327044,ENSG00000188976,2790
34,uc001adh.4,ENST00000263741,ENSG00000078808,2079
37,uc001ado.3,ENST00000450390,ENSG00000160087,2455
39,uc001aeb.2,ENST00000354700,ENSG00000131584,3896
41,uc001aee.2,ENST00000435064,ENSG00000127054,2148


In [10]:
df_best_transcripts_subset_once_red = df_best_transcripts_subset_once.drop(['name', 'transcript'], axis = 1, inplace = False)
df_best_transcripts_subset_once_red.head(5)

,gene,Length
16,ENSG00000188976,2790
34,ENSG00000078808,2079
37,ENSG00000160087,2455
39,ENSG00000131584,3896
41,ENSG00000127054,2148


In [11]:
print("Genes that appear only once in best transcripts database represent ", 5251*100/5440, "% of genes")
# Only 3.5% of genes are not completely defined

Genes that appear only once in best transcripts database represent  96.52573529411765 % of genes


In [12]:
# b) Genes that appear MORE THAN once in df_best_transcripts
df_best_transcripts_subset_more_than_once = df_best_transcripts_subset[df_best_transcripts_subset.duplicated(subset=['gene'], keep=False)]

print("Size: ", df_best_transcripts_subset_more_than_once.shape)
df_best_transcripts_subset_more_than_once.head()

Size:  (203, 4)


,name,transcript,gene,Length
90,uc001ajt.1,ENST00000463471,ENSG00000157873,4255
91,uc001ajr.3,ENST00000355716,ENSG00000157873,1707
502,uc009vtt.3,ENST00000373720,ENSG00000060688,1087
503,uc001bso.3,ENST00000263694,ENSG00000060688,1595
571,uc001byq.3,ENST00000468598,ENSG00000116560,2835


In [14]:
# For each gene, we have to choose which transcript length to use.
# In this case, we are going to select the maximum length between those transcripts that appear in Best_transcripts
# Before applying normalization, we will decide if we want to use these genes or not
df_best_transcripts_subset_more_than_once = df_best_transcripts_subset_more_than_once.groupby('gene').max()
print("Size: ", df_best_transcripts_subset_more_than_once.shape)
df_best_transcripts_subset_more_than_once.head()

Size:  (98, 3)


,name,transcript,Length
gene,,,
ENSG00000007392,uc010bqq.1,ENST00000490762,2643
ENSG00000011275,uc003spb.3,ENST00000476345,5639
ENSG00000011454,uc011lzk.3,ENST00000468760,4997
ENSG00000026297,uc003qvi.1,ENST00000508775,2445
ENSG00000033050,uc031szt.1,ENST00000287844,3546


In [15]:
list_of_genes_more_than_once = list(df_best_transcripts_subset_more_than_once.index)
len(list_of_genes_more_than_once)
# list_of_genes_more_than_once

98

In [16]:
df_best_transcripts_subset_more_than_once_red = df_best_transcripts_subset_more_than_once.drop(['name', 'transcript'], axis = 1, inplace = False)
df_best_transcripts_subset_more_than_once_red.reset_index(inplace=True)
df_best_transcripts_subset_more_than_once_red.head(5)

,gene,Length
0,ENSG00000007392,2643
1,ENSG00000011275,5639
2,ENSG00000011454,4997
3,ENSG00000026297,2445
4,ENSG00000033050,3546


In [17]:
# c) Genes that DO NOT appear in df_best_transcripts
list_of_best_transcripts_genes = list(df_best_transcripts['gene'])
len(list_of_best_transcripts_genes)
list_of_genes_not_appear = [x for x in list_of_genes if x not in list_of_best_transcripts_genes]
len(list_of_genes_not_appear)
# list_of_genes_not_appear

91

In [18]:
# For those genes, we have to choose which transcript length to use.
# In this case, we are going to use the maximum length between those transcripts that appear in GFF3
# Before applying normalization, we will decide if we want to use these genes or not
df_best_transcripts_subset_not_appear = df_GFF3[df_GFF3['gene'].isin(list_of_genes_not_appear)]
df_best_transcripts_subset_not_appear = df_best_transcripts_subset_not_appear.groupby('gene').max()

print("Size: ", df_best_transcripts_subset_not_appear.shape)
df_best_transcripts_subset_not_appear.head()

Size:  (91, 1)


,Length
gene,
ENSG00000066697,1991
ENSG00000104823,1411
ENSG00000113575,4649
ENSG00000128383,1478
ENSG00000136783,1633


In [19]:
df_best_transcripts_subset_not_appear_red = df_best_transcripts_subset_not_appear.reset_index()
df_best_transcripts_subset_not_appear_red.head(5)

,gene,Length
0,ENSG00000066697,1991
1,ENSG00000104823,1411
2,ENSG00000113575,4649
3,ENSG00000128383,1478
4,ENSG00000136783,1633


In [19]:
# Quick check
# Gene ENSG00000066697
# From gencode.v19.annotation it has the following transcripts:
# transcript	ID=ENST00000395067
# transcript	ID=ENST00000398977
# transcript	ID=ENST00000374885
# transcript	ID=ENST00000374886
# transcript	ID=ENST00000489377

# From knowToEnsemble
# transcript	ID=ENST00000395067      uc022bkz
# transcript	ID=ENST00000395067      uc004baw
# transcript	ID=ENST00000395067      uc022bla
# transcript	ID=ENST00000395067      uc022blb

# transcript	ID=ENST00000398977      Not Found
# transcript	ID=ENST00000374885      Not Found
# transcript	ID=ENST00000374886      Not Found

# transcript	ID=ENST00000489377      uc004bax

# From knownCanonical
# transcript	ID=ENST00000395067      uc022bkz       Not Found  
# transcript	ID=ENST00000395067      uc004baw       Not Found
# transcript	ID=ENST00000395067      uc022bla       Not Found
# transcript	ID=ENST00000395067      uc022blb       Not Found

# transcript	ID=ENST00000489377      uc004bax       Not Found

In [21]:
# Create a unique dataframe with all genes we want to use and lengths
# In this case, we keep only those genes that have a best transcript
# df_genes_length = pd.concat([df_best_transcripts_subset_once_red,df_best_transcripts_subset_more_than_once_red,df_best_transcripts_subset_not_appear_red], axis=0)
df_genes_length = df_best_transcripts_subset_once_red.copy(deep=True)
print("Size: ", df_genes_length.shape)  # 5440 - 91 - 98 = 5251
df_genes_length.head(3)

Size:  (5251, 2)


,gene,Length
16,ENSG00000188976,2790
34,ENSG00000078808,2079
37,ENSG00000160087,2455


In [22]:
# Get the complete list of genes that appeared more than once or zero times in best transcripts list
# And save it
list_of_genes_to_check = list_of_genes_not_appear + list_of_genes_more_than_once
len(list_of_genes_to_check)
df = pd.DataFrame(list_of_genes_to_check)
df.to_csv('List_of_genes_to_check.csv', index=False, header=None)

### Apply Normalization

In [23]:
# Verify there is no transcript with length zero
print((df_genes_length == 0).all())

gene      False
Length    False
dtype: bool


In [24]:
df_data.shape

(5440, 2351)

In [25]:
df_data.head(3)

,1-Vumc-HD-101-TR922,2-Vumc-HD-103-TR923,3-Vumc-HD-108-TR924,4-Vumc-HD-127-TR925,5-Vumc-HD-130-TR926,6-Vumc-HD-142-TR927,7-Vumc-HD-143-TR928,8-Vumc-HD-144-TR929,9-Vumc-HD-145-TR930,11-Vumc-HD-149-TR932,...,countMatrix.3502-NKI-4854-SARC-TR3080,countMatrix.3503-NKI-4707-SARC-TR3079,countMatrix.3504-NKI-4941-SARC-TR3084,countMatrix.3507-NKI-6348-SARC-TR3134,countMatrix.3508-NKI-5275-SARC-TR3099,countMatrix.3512-NKI-5148-SARC-TR3129,countMatrix.320-10,countMatrix.327-34,countMatrix.376-P-PH-001-013-TR1400,countMatrix.506-NKI-NSCLC-107-270
ENSG00000000419,5,79,58,128,93,12,0,0,18,28,...,3,0,1,2,9,16,22,21,0,1
ENSG00000000460,8,10,21,14,6,8,25,42,5,7,...,0,3,0,2,63,3,3,0,1,0
ENSG00000000938,24,336,334,59,120,22,31,13,19,36,...,8,45,1,7,163,90,46,75,14,8


In [26]:
df_genes_length.index=df_genes_length['gene']
df_genes_length.drop(['gene'], axis=1, inplace=True)
df_genes_length.head(3)


,Length
gene,
ENSG00000188976,2790
ENSG00000078808,2079
ENSG00000160087,2455


In [27]:
# Add information about length in df_data
# Merge: inner (keep only genes that appear in both databases)
df_data_wLength = pd.merge(df_data, df_genes_length, left_index = True, right_index = True, how='inner')
print("Size: ", df_data_wLength.shape)
df_data_wLength.head(3)

Size:  (5251, 2352)


,1-Vumc-HD-101-TR922,2-Vumc-HD-103-TR923,3-Vumc-HD-108-TR924,4-Vumc-HD-127-TR925,5-Vumc-HD-130-TR926,6-Vumc-HD-142-TR927,7-Vumc-HD-143-TR928,8-Vumc-HD-144-TR929,9-Vumc-HD-145-TR930,11-Vumc-HD-149-TR932,...,countMatrix.3503-NKI-4707-SARC-TR3079,countMatrix.3504-NKI-4941-SARC-TR3084,countMatrix.3507-NKI-6348-SARC-TR3134,countMatrix.3508-NKI-5275-SARC-TR3099,countMatrix.3512-NKI-5148-SARC-TR3129,countMatrix.320-10,countMatrix.327-34,countMatrix.376-P-PH-001-013-TR1400,countMatrix.506-NKI-NSCLC-107-270,Length
ENSG00000000419,5,79,58,128,93,12,0,0,18,28,...,0,1,2,9,16,22,21,0,1,1075
ENSG00000000460,8,10,21,14,6,8,25,42,5,7,...,3,0,2,63,3,3,0,1,0,4355
ENSG00000000938,24,336,334,59,120,22,31,13,19,36,...,45,1,7,163,90,46,75,14,8,2729


In [28]:
# Apply TPM
# Step 1: Normalize for gene length (Kb)
# For each row, divide each element by transcript length
col_names = list(df_data_wLength.columns)
df_data_RFK = 1000*df_data_wLength[col_names[0:-1]].div(df_data_wLength[col_names[-1]], axis = 0)

df_data_RFK.head(5)

,1-Vumc-HD-101-TR922,2-Vumc-HD-103-TR923,3-Vumc-HD-108-TR924,4-Vumc-HD-127-TR925,5-Vumc-HD-130-TR926,6-Vumc-HD-142-TR927,7-Vumc-HD-143-TR928,8-Vumc-HD-144-TR929,9-Vumc-HD-145-TR930,11-Vumc-HD-149-TR932,...,countMatrix.3502-NKI-4854-SARC-TR3080,countMatrix.3503-NKI-4707-SARC-TR3079,countMatrix.3504-NKI-4941-SARC-TR3084,countMatrix.3507-NKI-6348-SARC-TR3134,countMatrix.3508-NKI-5275-SARC-TR3099,countMatrix.3512-NKI-5148-SARC-TR3129,countMatrix.320-10,countMatrix.327-34,countMatrix.376-P-PH-001-013-TR1400,countMatrix.506-NKI-NSCLC-107-270
ENSG00000000419,4.651163,73.488372,53.953488,119.069767,86.511628,11.162791,0.000000,0.000000,16.744186,26.046512,...,2.790698,0.000000,0.930233,1.860465,8.372093,14.883721,20.465116,19.534884,0.000000,0.930233
ENSG00000000460,1.836969,2.296211,4.822044,3.214696,1.377727,1.836969,5.740528,9.644087,1.148106,1.607348,...,0.000000,0.688863,0.000000,0.459242,14.466131,0.688863,0.688863,0.000000,0.229621,0.000000
ENSG00000000938,8.794430,123.122023,122.389154,21.619641,43.972151,8.061561,11.359472,4.763650,6.962257,13.191645,...,2.931477,16.489557,0.366435,2.565042,59.728838,32.979113,16.855991,27.482594,5.130084,2.931477
ENSG00000001036,16.129032,19.524618,6.366723,15.280136,10.611205,18.251273,11.460102,11.460102,25.042445,2.122241,...,0.424448,1.273345,1.697793,6.366723,1.273345,2.546689,4.244482,3.395586,1.273345,0.000000
ENSG00000001461,13.136289,3.101624,2.006933,1.642036,3.284072,5.108557,0.912242,1.094691,8.939974,3.284072,...,0.182448,0.182448,0.000000,0.000000,2.189381,0.364897,1.094691,1.094691,0.182448,0.000000


In [29]:
# Step 2: Normalize for sequencing depth (M)
# For each column: 
#    sum all entries and divide by 10^6
#    then, divide each element by the above number

# Total = data_df_present_wLength.sum()
# Total = Total / 10**6

df_data_TPM = df_data_RFK.transform(lambda x: (x*10**6)/x.sum())

df_data_TPM.head(5)

,1-Vumc-HD-101-TR922,2-Vumc-HD-103-TR923,3-Vumc-HD-108-TR924,4-Vumc-HD-127-TR925,5-Vumc-HD-130-TR926,6-Vumc-HD-142-TR927,7-Vumc-HD-143-TR928,8-Vumc-HD-144-TR929,9-Vumc-HD-145-TR930,11-Vumc-HD-149-TR932,...,countMatrix.3502-NKI-4854-SARC-TR3080,countMatrix.3503-NKI-4707-SARC-TR3079,countMatrix.3504-NKI-4941-SARC-TR3084,countMatrix.3507-NKI-6348-SARC-TR3134,countMatrix.3508-NKI-5275-SARC-TR3099,countMatrix.3512-NKI-5148-SARC-TR3129,countMatrix.320-10,countMatrix.327-34,countMatrix.376-P-PH-001-013-TR1400,countMatrix.506-NKI-NSCLC-107-270
ENSG00000000419,3.937800,67.841060,82.445588,144.198227,85.057059,9.972581,0.000000,0.000000,12.591351,31.777072,...,100.611799,0.000000,27.876966,7.352040,42.557469,102.610884,113.742517,109.494730,0.000000,20.438796
ENSG00000000460,1.555228,2.119756,7.368499,3.893125,1.354562,1.641106,4.863694,10.302517,0.863356,1.960985,...,0.000000,10.382223,0.000000,1.814797,73.535006,4.749140,3.828615,0.000000,2.622014,0.000000
ENSG00000000938,7.445603,113.660546,187.021192,26.182246,43.232822,7.202014,9.624375,5.088878,5.235502,16.093973,...,105.687245,248.522789,10.981216,10.136332,303.616808,227.363572,93.683458,154.042343,58.579768,64.409543
ENSG00000001036,13.655275,18.024223,9.728903,18.504853,10.432793,16.305269,9.709634,12.242516,18.831504,2.589161,...,15.302445,19.191248,50.879012,25.159517,6.472733,17.557306,23.590293,19.032555,14.540158,0.000000
ENSG00000001461,11.121538,2.863275,3.066767,1.988571,3.228855,4.563867,0.772902,1.169428,6.722713,4.006609,...,6.577734,2.749777,0.000000,0.000000,11.129180,2.515661,6.084152,6.135837,2.083355,0.000000


In [30]:
df_data_TPM.shape

(5251, 2351)

In [31]:
# Verify TPM is correctly done by checking each patient (column) adds up 1M
df_data_TPM.sum()

1-Vumc-HD-101-TR922                      1000000.0
2-Vumc-HD-103-TR923                      1000000.0
3-Vumc-HD-108-TR924                      1000000.0
4-Vumc-HD-127-TR925                      1000000.0
5-Vumc-HD-130-TR926                      1000000.0
                                           ...    
countMatrix.3512-NKI-5148-SARC-TR3129    1000000.0
countMatrix.320-10                       1000000.0
countMatrix.327-34                       1000000.0
countMatrix.376-P-PH-001-013-TR1400      1000000.0
countMatrix.506-NKI-NSCLC-107-270        1000000.0
Length: 2351, dtype: float64

## Transpose

In [32]:
df_data = df_data_TPM.T

In [33]:
display(df_data.head(5))

,ENSG00000000419,ENSG00000000460,ENSG00000000938,ENSG00000001036,ENSG00000001461,ENSG00000001497,ENSG00000001629,ENSG00000001631,ENSG00000002330,ENSG00000002549,...,ENSG00000257267,ENSG00000257923,ENSG00000258890,ENSG00000261253,ENSG00000263563,ENSG00000264538,ENSG00000265148,ENSG00000266714,ENSG00000267243,ENSG00000271043
1-Vumc-HD-101-TR922,3.937800,1.555228,7.445603,13.655275,11.121538,3.471206,5.986674,4.978068,46.477912,25.632179,...,69.065205,66.636392,4.045685,180.566933,26.692834,24.964644,53.934968,0.539998,32.907766,284.930419
2-Vumc-HD-103-TR923,67.841060,2.119756,113.660546,18.024223,2.863275,15.139871,3.036190,18.222689,40.741981,42.549333,...,30.267333,14.505070,26.468248,59.756005,23.411025,10.651773,32.718322,11.972440,16.903231,118.398686
3-Vumc-HD-108-TR924,82.445588,7.368499,187.021192,9.728903,3.066767,7.518260,23.872422,11.552102,24.673084,66.978931,...,24.454153,28.678713,18.498656,97.645350,25.136444,9.795425,61.013765,24.528653,6.381343,145.882880
4-Vumc-HD-127-TR925,144.198227,3.893125,26.182246,18.504853,1.988571,6.454907,3.385574,10.172531,33.893471,22.436797,...,21.507537,5.021406,18.518608,25.125307,19.091100,19.407689,10.866216,18.667032,1.556106,101.601218
5-Vumc-HD-130-TR926,85.057059,1.354562,43.232822,10.432793,3.228855,3.627994,2.748589,10.323251,39.158125,47.981986,...,42.213313,6.222246,20.672286,25.293593,20.890185,8.193220,19.407897,1.254198,0.947497,162.126211


## Look for Nan Values


In [34]:
print(df_data.shape)
genes = df_data.columns
pacients = df_data.index
print("Genes: ", genes[:10])
print("Pacients: ", pacients[:10])

(2351, 5251)
Genes:  Index(['ENSG00000000419', 'ENSG00000000460', 'ENSG00000000938',
       'ENSG00000001036', 'ENSG00000001461', 'ENSG00000001497',
       'ENSG00000001629', 'ENSG00000001631', 'ENSG00000002330',
       'ENSG00000002549'],
      dtype='object')
Pacients:  Index(['1-Vumc-HD-101-TR922', '2-Vumc-HD-103-TR923', '3-Vumc-HD-108-TR924',
       '4-Vumc-HD-127-TR925', '5-Vumc-HD-130-TR926', '6-Vumc-HD-142-TR927',
       '7-Vumc-HD-143-TR928', '8-Vumc-HD-144-TR929', '9-Vumc-HD-145-TR930',
       '11-Vumc-HD-149-TR932'],
      dtype='object')


In [35]:
# Verify data type, Naans etc
# print(df_data.info())
# print(df_data.describe())

In [36]:
df_data.dtypes
df_data.isnull().values.any()
# df_data.isnull().sum()

False

## Create DataFrame from metadata (ground truth)

### Txt

In [37]:
# Load txt file
file = 'IntVeld2022_metadata.txt'

with open (file, "r") as f:
    y_data = f.read().splitlines()

In [38]:
# Change names to match those in the excel file
ys_data = []
line = 0

Group_txt = ['Asymptomatic Controls', 'Pancreatic Disease', 'Urothelial Carcinoma', 'Epilepsy', 'Esophageal Cancer', 'Renal Cell Carcinoma', 'Hodgkin Lymphoma']
Group_xml = ['Asymptomatic controls', 'Pancreatic diseases', 'Urothelial cancer', 'Medically-intractable epilepsy', 'Esophageal carcinoma', 'Renal Cell cancer', 'Lymphoma']


while line < len(y_data):
    if y_data[line]:
        if y_data[line][0].isdigit(): 
            Patient = re.split('[0-9]+: ', y_data[line])[1]
            Class_group = re.split('"',y_data[line+6])[1]
            Patient_group = re.split('"',y_data[line+7])[1]
            if Patient_group in Group_txt:
                Patient_group = Group_xml[Group_txt.index(Patient_group)]
            ys_data.append([Patient,Class_group,Patient_group])
            line += 9
    line += 1

print(len(ys_data))
print(ys_data[0])

2351
['NKI-NSCLC-4861-TR2958', 'Malignant', 'Non-Small-Cell Lung Cancer']


In [39]:
df_y = pd.DataFrame(ys_data, columns=['Sample ID', 'Class_group','Patient_group'])
df_y.sample(5)

,Sample ID,Class_group,Patient_group
19,TR3660-OVA-CATH,Malignant,Ovarian Cancer
1266,TR4253-HN-VUMC-BB,Malignant,Head and Neck Cancer
193,UMCG-NSCLC-1097-TR2856,Malignant,Non-Small-Cell Lung Cancer
1093,Vumc-MS-Presto-021-TR1429,nonMalignant,Multiple Sclerosis
673,Vumc-GBM-follow-up-VU501t0,Malignant,Glioma


In [40]:
df_y.shape

(2351, 3)

In [41]:
# Check values per class

Class_group_values = df_y['Class_group'].unique()
print("Class_group_values: ", Class_group_values)

Patient_group_values_Malignant = df_y.loc[df_y['Class_group'] == 'Malignant','Patient_group'].unique()
print("\nPatient_group_values_Malignant: ", sorted(list(Patient_group_values_Malignant)))
print("Total Patient_group_values_Malignant: ", len(Patient_group_values_Malignant))
print("Number of Malignant samples: " , len(df_y.loc[df_y['Class_group'] == 'Malignant']))

Patient_group_values_nonMalignant = df_y.loc[df_y['Class_group'] == 'nonMalignant','Patient_group'].unique()
print("\nPatient_group_values_nonMalignant: ", sorted(list(Patient_group_values_nonMalignant)))
print("Total Patient_group_values_nonMalignant: ", len(Patient_group_values_nonMalignant))
print("Number of non Malignant samples: " , len(df_y.loc[df_y['Class_group'] == 'nonMalignant']))

Class_group_values:  ['Malignant' 'nonMalignant']

Patient_group_values_Malignant:  ['Asymptomatic controls', 'Breast Cancer', 'Cholangiocarcinoma', 'Colorectal Cancer', 'Endometrial Cancer', 'Esophageal carcinoma', 'Glioma', 'Head and Neck Cancer', 'Hepatocellular Carcinoma', 'Lymphoma', 'Melanoma', 'Multiple Myeloma', 'Non-Small-Cell Lung Cancer', 'Ovarian Cancer', 'Pancreatic Cancer', 'Prostate Cancer', 'Renal Cell cancer', 'Sarcoma', 'Urothelial cancer']
Total Patient_group_values_Malignant:  19
Number of Malignant samples:  1648

Patient_group_values_nonMalignant:  ['Angina Pectoris', 'Asymptomatic controls', 'Bowel Disease', 'Former Sarcoma', 'Hematuria', 'Medically-intractable epilepsy', 'Multiple Sclerosis', 'Pancreatic diseases', 'Prostate Cancer', 'Pulmonary Hypertension', 'nSTEMI']
Total Patient_group_values_nonMalignant:  11
Number of non Malignant samples:  703


In [42]:
# Find incorrect data that does not match info from the article
df_filtered_y = df_y[(df_y['Class_group']=='Malignant') & (df_y['Patient_group']=='Asymptomatic controls') ]
print("Number of samples Malignant + Asymtomatic controls: ", len(df_filtered_y))

Number of samples Malignant + Asymtomatic controls:  21


In [43]:
df_filtered_y = df_y[(df_y['Class_group']=='nonMalignant') & (df_y['Patient_group']=='Prostate Cancer') ]
print("Number of samples nonMalignant + Prostate Cancer: ", len(df_filtered_y))

Number of samples nonMalignant + Prostate Cancer:  1


In [44]:
# Correct incorrect data
df_y.loc[(df_y['Class_group']=='nonMalignant') & (df_y['Patient_group'] == 'Prostate Cancer'), 'Class_group'] = 'Malignant'
df_y.loc[(df_y['Class_group']=='Malignant') & (df_y['Patient_group'] == 'Asymptomatic controls'), 'Class_group'] = 'nonMalignant'


In [45]:
print("Number of Malignant samples: " , len(df_y.loc[df_y['Class_group'] == 'Malignant']))
print("Number of non Malignant samples: " , len(df_y.loc[df_y['Class_group'] == 'nonMalignant']))

Number of Malignant samples:  1628
Number of non Malignant samples:  723


In [46]:
print(df_y.loc[df_y['Class_group'] == 'Malignant'].groupby(by='Patient_group').count())

                            Sample ID  Class_group
Patient_group                                     
Breast Cancer                      93           93
Cholangiocarcinoma                 85           85
Colorectal Cancer                  85           85
Endometrial Cancer                 39           39
Esophageal carcinoma               15           15
Glioma                            132          132
Head and Neck Cancer              101          101
Hepatocellular Carcinoma           23           23
Lymphoma                           20           20
Melanoma                           68           68
Multiple Myeloma                   31           31
Non-Small-Cell Lung Cancer        522          522
Ovarian Cancer                    144          144
Pancreatic Cancer                 126          126
Prostate Cancer                    35           35
Renal Cell cancer                  28           28
Sarcoma                            53           53
Urothelial cancer              

In [47]:
print(df_y.loc[df_y['Class_group'] == 'nonMalignant'].groupby(by='Patient_group').count())

                                Sample ID  Class_group
Patient_group                                         
Angina Pectoris                        26           26
Asymptomatic controls                 391          391
Bowel Disease                           9            9
Former Sarcoma                         26           26
Hematuria                               5            5
Medically-intractable epilepsy         44           44
Multiple Sclerosis                     86           86
Pancreatic diseases                    50           50
Pulmonary Hypertension                 71           71
nSTEMI                                 15           15


### XLSX

In [48]:
# Add information about age, sex, cancer stage and supplying institutions
# Compare data between xlsx and txt and correct some information

file = '1-s2.0-S1535610822003701-mmc3.xlsx'
df_y_xlsx = pd.read_excel(file, header=2)

In [49]:
df_y_xlsx.head(5)

,Sample ID,Group,Stage,Sex,Age,Sample-supplying institution,Training series,Evaluation series,Validation series,lib.size,classificationScoreCancer
0,Vumc-HD-101-TR922,Asymptomatic controls,n.a.,F,35,Institute 13,1,0,0,2296824,0.485949
1,Vumc-HD-103-TR923,Asymptomatic controls,n.a.,F,54,Institute 13,0,1,0,1731594,0.370735
2,Vumc-HD-108-TR924,Asymptomatic controls,n.a.,F,56,Institute 13,0,0,1,1136912,0.799796
3,Vumc-HD-127-TR925,Asymptomatic controls,n.a.,F,52,Institute 13,1,0,0,1115322,0.580309
4,Vumc-HD-130-TR926,Asymptomatic controls,n.a.,F,57,Institute 13,0,1,0,1331137,0.402155


In [50]:
df_y_xlsx.replace('n.a.', np.nan, inplace=True)

In [51]:
df_y_xlsx.isnull().values.any()

True

In [52]:
# Check which columns have Nan values
list_na_columns = df_y_xlsx.columns[df_y_xlsx.isna().any()].tolist()
print(list_na_columns)


['Stage', 'Sex', 'Age']


In [53]:
# Merge info from txt and xmls
df_y_merged = pd.merge(df_y_xlsx, df_y, on=['Sample ID'], how='inner')
df_y_merged.shape

(2351, 13)

In [54]:
df_y_merged.head(5)

,Sample ID,Group,Stage,Sex,Age,Sample-supplying institution,Training series,Evaluation series,Validation series,lib.size,classificationScoreCancer,Class_group,Patient_group
0,Vumc-HD-101-TR922,Asymptomatic controls,NaN,F,35.0,Institute 13,1,0,0,2296824,0.485949,nonMalignant,Asymptomatic controls
1,Vumc-HD-103-TR923,Asymptomatic controls,NaN,F,54.0,Institute 13,0,1,0,1731594,0.370735,nonMalignant,Asymptomatic controls
2,Vumc-HD-108-TR924,Asymptomatic controls,NaN,F,56.0,Institute 13,0,0,1,1136912,0.799796,nonMalignant,Asymptomatic controls
3,Vumc-HD-127-TR925,Asymptomatic controls,NaN,F,52.0,Institute 13,1,0,0,1115322,0.580309,nonMalignant,Asymptomatic controls
4,Vumc-HD-130-TR926,Asymptomatic controls,NaN,F,57.0,Institute 13,0,1,0,1331137,0.402155,nonMalignant,Asymptomatic controls


In [55]:
# Find differences and change values
df_y_merged[df_y_merged["Group"].astype(str).str.lower() != df_y_merged["Patient_group"].astype(str).str.lower()]

,Sample ID,Group,Stage,Sex,Age,Sample-supplying institution,Training series,Evaluation series,Validation series,lib.size,classificationScoreCancer,Class_group,Patient_group
2318,Vumc-P-PH-001-024-TR1414,Pulmonary Hypertension,NaN,M,71.0,Institute 13,0,0,1,464882,0.949583,nonMalignant,Asymptomatic controls


In [56]:
df_y_merged.loc[df_y_merged["Group"].astype(str).str.lower() != df_y_merged["Patient_group"].astype(str).str.lower(),'Patient_group'] = 'Pulmonary Hypertension'

In [57]:
# Review numbers and compare with article
df_y_merged.groupby('Class_group').size()

Class_group
Malignant       1628
nonMalignant     723
dtype: int64

In [58]:
types_of_cancers = df_y_merged[df_y_merged['Class_group'] == 'Malignant'].groupby(by='Patient_group').size()
print(types_of_cancers)
print("Total number of cancers: ", len(types_of_cancers))

Patient_group
Breast Cancer                  93
Cholangiocarcinoma             85
Colorectal Cancer              85
Endometrial Cancer             39
Esophageal carcinoma           15
Glioma                        132
Head and Neck Cancer          101
Hepatocellular Carcinoma       23
Lymphoma                       20
Melanoma                       68
Multiple Myeloma               31
Non-Small-Cell Lung Cancer    522
Ovarian Cancer                144
Pancreatic Cancer             126
Prostate Cancer                35
Renal Cell cancer              28
Sarcoma                        53
Urothelial cancer              28
dtype: int64
Total number of cancers:  18


In [59]:
types_of_nonCancers = df_y_merged[df_y_merged['Class_group'] == 'nonMalignant'].groupby(by='Patient_group').size()
print(types_of_nonCancers)
print("Total number of categories of nonMalignant patients (apart from Asymptomatic controls): ", len(types_of_nonCancers) - 1)

Patient_group
Angina Pectoris                    26
Asymptomatic controls             390
Bowel Disease                       9
Former Sarcoma                     26
Hematuria                           5
Medically-intractable epilepsy     44
Multiple Sclerosis                 86
Pancreatic diseases                50
Pulmonary Hypertension             72
nSTEMI                             15
dtype: int64
Total number of categories of nonMalignant patients (apart from Asymptomatic controls):  9


In [60]:
# Article: For a subset of samples, tumor stages were unknown (n.a.; n = 124) or not informative (n.i.; e.g.,
# gliomas and multiple myeloma; n = 132; in total n = 256; 16% of all cancers).
stage_of_cancers = df_y_merged[df_y_merged['Class_group'] == 'Malignant'].groupby(by='Stage').size()
print(stage_of_cancers)

stage_of_cancers_glioma = df_y_merged[df_y_merged['Patient_group'] == 'Glioma'].groupby(by='Stage').size()
print("\n",stage_of_cancers_glioma)


Stage
I      111
II     166
III    235
IV     860
dtype: int64

 Series([], dtype: int64)


In [61]:
# Correct data: nonMalignant should not have Stage
df_y_merged.loc[(~df_y_merged['Stage'].isna()) & (df_y_merged['Class_group']=='nonMalignant')]


,Sample ID,Group,Stage,Sex,Age,Sample-supplying institution,Training series,Evaluation series,Validation series,lib.size,classificationScoreCancer,Class_group,Patient_group
789,Vumc-ChronPan-29-TR1045,Pancreatic diseases,IV,M,58.0,Institute 13,0,0,1,1504224,0.946398,nonMalignant,Pancreatic diseases


In [62]:
df_y_merged.loc[(~df_y_merged['Stage'].isna()) & (df_y_merged['Class_group']=='nonMalignant'),'Stage'] = np.nan

In [63]:
# Remove columns that are repeated or irrelevant
df_y_merged.drop('Group', axis=1, inplace=True)
df_y_merged.drop('lib.size', axis=1, inplace=True)
df_y_merged.drop('classificationScoreCancer', axis=1, inplace=True)
df_y_merged.sample(5)

,Sample ID,Stage,Sex,Age,Sample-supplying institution,Training series,Evaluation series,Validation series,Class_group,Patient_group
2196,TR4547-SARC-NKI,IV,F,78.0,Institute 5,0,1,0,Malignant,Sarcoma
1503,Vumc-NSCLC-374-TR2178,IV,F,56.0,Institute 13,0,0,1,Malignant,Non-Small-Cell Lung Cancer
689,MGH-BrCa-P12-TR703,IV,F,43.0,Institute 4,0,0,1,Malignant,Breast Cancer
1061,TR4808-NSCLC-MAAS,NaN,M,76.0,Institute 3,0,0,1,nonMalignant,Asymptomatic controls
86,Vumc-HD-21-1-TR1161,NaN,F,39.0,Institute 13,1,0,0,nonMalignant,Asymptomatic controls


In [64]:
df_y_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2351 entries, 0 to 2350
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Sample ID                     2351 non-null   object 
 1   Stage                         1372 non-null   object 
 2   Sex                           2331 non-null   object 
 3   Age                           2328 non-null   float64
 4   Sample-supplying institution  2351 non-null   object 
 5   Training series               2351 non-null   int64  
 6   Evaluation series             2351 non-null   int64  
 7   Validation series             2351 non-null   int64  
 8   Class_group                   2351 non-null   object 
 9   Patient_group                 2351 non-null   object 
dtypes: float64(1), int64(3), object(6)
memory usage: 202.0+ KB


## Combine DataFrames containing info about platelets and ground truth


In [65]:
# Check patients names
patients_d = list(df_data.index)
print(patients_d[:10])
print(len(patients_d))

patients_y = list(df_y_merged['Sample ID'])
print(patients_y[:10])
print(len(patients_y))

['1-Vumc-HD-101-TR922', '2-Vumc-HD-103-TR923', '3-Vumc-HD-108-TR924', '4-Vumc-HD-127-TR925', '5-Vumc-HD-130-TR926', '6-Vumc-HD-142-TR927', '7-Vumc-HD-143-TR928', '8-Vumc-HD-144-TR929', '9-Vumc-HD-145-TR930', '11-Vumc-HD-149-TR932']
2351
['Vumc-HD-101-TR922', 'Vumc-HD-103-TR923', 'Vumc-HD-108-TR924', 'Vumc-HD-127-TR925', 'Vumc-HD-130-TR926', 'Vumc-HD-142-TR927', 'Vumc-HD-143-TR928', 'Vumc-HD-144-TR929', 'Vumc-HD-145-TR930', 'Vumc-HD-149-TR932']
2351


In [66]:
compare_patients = [ x.find(y)!=-1 for (x,y) in zip(patients_d, patients_y)]
print("Number of patients: ", len(compare_patients), "Number of name coincidences",  sum(compare_patients), "For example: ", patients_d[0], " and ", patients_y[0])

print("\nCases where names are not simmilar: ")
different_patients =  [ x.find(y)==-1 for (x,y) in zip(patients_d, patients_y)]
different_patients_index = [i for i, x in enumerate(different_patients) if x]
for idx in different_patients_index:
    print(patients_d[idx], "and ", end='')
    print(patients_y[idx])

Number of patients:  2351 Number of name coincidences 2348 For example:  1-Vumc-HD-101-TR922  and  Vumc-HD-101-TR922

Cases where names are not simmilar: 
countMatrix.320-10 and Vumc-P-PH-001-010-TR1505
countMatrix.327-34 and Vumc-P-PH-001-034-TR1512
countMatrix.506-NKI-NSCLC-107-270 and NKI-NSCLC-107-270-TR725


In [67]:
# We assume both databases have same list of patients and in the same order.
# Therefore, we can join them

df_y_merged.index = patients_d
df = pd.merge(df_data, df_y_merged, left_index=True, right_index=True, how='inner')

In [68]:
# df.head(5)
# display(df)
print(df.shape)

(2351, 5261)


In [69]:
# Check which columns have Nan values
list_na_columns = df.columns[df.isna().any()].tolist()
print(list_na_columns)

# df['Age'].describe()
# df['Age'].dtype       # float64
# df.loc[df['Age'] == 18]

# df['Age'].isnull().values.any()
# df.isnull().values.any()

# df['Stage'].isna().sum()
# df['Stage'].describe()

print(df['Stage'].isnull().sum())
print(df['Sex'].isnull().sum())
print(df['Age'].isnull().sum())

['Stage', 'Sex', 'Age']
979
20
23


In [70]:
# Save merged dataFrame
file='DataPatientsWithGT.csv'
df.to_csv(file, sep=';', na_rep='n.a.')

In [71]:
df.head(3)

,ENSG00000000419,ENSG00000000460,ENSG00000000938,ENSG00000001036,ENSG00000001461,ENSG00000001497,ENSG00000001629,ENSG00000001631,ENSG00000002330,ENSG00000002549,...,Sample ID,Stage,Sex,Age,Sample-supplying institution,Training series,Evaluation series,Validation series,Class_group,Patient_group
1-Vumc-HD-101-TR922,3.937800,1.555228,7.445603,13.655275,11.121538,3.471206,5.986674,4.978068,46.477912,25.632179,...,Vumc-HD-101-TR922,NaN,F,35.0,Institute 13,1,0,0,nonMalignant,Asymptomatic controls
2-Vumc-HD-103-TR923,67.841060,2.119756,113.660546,18.024223,2.863275,15.139871,3.036190,18.222689,40.741981,42.549333,...,Vumc-HD-103-TR923,NaN,F,54.0,Institute 13,0,1,0,nonMalignant,Asymptomatic controls
3-Vumc-HD-108-TR924,82.445588,7.368499,187.021192,9.728903,3.066767,7.518260,23.872422,11.552102,24.673084,66.978931,...,Vumc-HD-108-TR924,NaN,F,56.0,Institute 13,0,0,1,nonMalignant,Asymptomatic controls
